This notebook will use Keras to implement a nueral network structure proposed by the paper <br>
***Next Basket Recommendation with Neural Networks*** <br>
http://ceur-ws.org/Vol-1441/recsys2015_poster15.pdf

## package

In [1]:
# import packages
import os
import sys
import numpy as np
import pandas as pd

from string import punctuation
from collections import defaultdict
import ast

from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, Embedding, Dropout, Activation, merge, LSTM
from keras.layers.core import Lambda
from keras.layers.merge import concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
from keras import metrics
from keras import regularizers
import tensorflow as tf

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MultiLabelBinarizer

import sys

Using TensorFlow backend.


## Load data and define parameters and directory

In [2]:
# define constants and parameters
INPUT_DIR = '../input/'
TRANSFORM_DIR = "../transform/"
OUTOUT_DIR = "../output/"
PRIOR_DATA_FILE = TRANSFORM_DIR + 'orders_prior_merge.csv'
TRAIN_DATA_FILE = TRANSFORM_DIR + 'orders_train_merge.csv'
TEST_DATA_FILE = TRANSFORM_DIR + 'orders_test.csv'

VALIDATION_SPLIT_RATIO = 0.1

In [3]:
# load data
orders_train_merge = pd.read_csv(TRAIN_DATA_FILE)
orders_prior_merge = pd.read_csv(PRIOR_DATA_FILE)
orders_test = pd.read_csv(TEST_DATA_FILE)
products = pd.read_csv(INPUT_DIR+"products.csv")

# define constants and parameters
MAX_ITEM = max(max(orders_train_merge["product_id"].apply(lambda x:len(ast.literal_eval(x)))), 
               max(orders_prior_merge["product_id"].apply(lambda x:len(ast.literal_eval(x)))))
PRE_BASKET = min(min(orders_train_merge.order_number), min(orders_test.order_number))-1

KeyboardInterrupt: 

In [ ]:
num_dense = np.random.randint(100, 120)
rate_drop_dense = 0.15 + np.random.rand() * 0.25

STAMP = 'nn_%d_%.2f'%(num_dense, rate_drop_dense)

## Train and test data

In [ ]:
# get useful columns
prior = orders_prior_merge[["order_id","user_id","product_id"]]
train = orders_train_merge[["order_id","user_id","product_id"]]
test = orders_test[["order_id","user_id"]]

# create user_id dict
train_usr_dict = dict.fromkeys(train.user_id)
test_usr_dict = dict.fromkeys(test.user_id)

# split train and test prior data
prior_train = prior.ix[[x in train_usr_dict for x in list(prior.user_id)],:]
prior_test = prior.ix[[x in test_usr_dict for x in list(prior.user_id)],:]

## Train Validation split

In [ ]:
# Train Validation split
def split_train_val(data, val_ratio):
    shuffled_indices = np.random.permutation(len(data))
    val_set_size = int(len(data)*val_ratio)
    val_indices = shuffled_indices[:val_set_size]
    train_indices = shuffled_indices[val_set_size:]
    return data.iloc[train_indices], data.iloc[val_indices]

np.random.seed(1992)
train_train, train_val = split_train_val(train, VALIDATION_SPLIT_RATIO)

# create user_id dict
train_train_usr_dict = dict.fromkeys(train_train.user_id)
train_val_usr_dict = dict.fromkeys(train_val.user_id)

# split train_train and train_val prior data
prior_train_train = prior_train.ix[[x in train_train_usr_dict for x in list(prior_train.user_id)],:]
prior_train_val = prior_train.ix[[x in train_val_usr_dict for x in list(prior_train.user_id)],:]

## Data processing for model training

In [ ]:
# prior
def get_pre_baskets(prior):
    return prior.groupby("user_id").tail(3)

train_train_baskets = get_pre_baskets(prior_train_train)
train_val_baskets = get_pre_baskets(prior_train_val)

test_baskets = get_pre_baskets(prior_test)

assert min(train_val_baskets.groupby("user_id").apply(len)) == PRE_BASKET
assert min(train_train_baskets.groupby("user_id").apply(len)) == PRE_BASKET

# extract sequence of items from pandas frame
sequence_train_train = list(map(ast.literal_eval,train_train_baskets["product_id"].tolist()))
sequence_train_val = list(map(ast.literal_eval,train_val_baskets["product_id"].tolist()))

sequence_test = list(map(ast.literal_eval,test_baskets["product_id"].tolist()))

# pad all train and val with max items
data_train_train = pad_sequences(sequence_train_train, maxlen=MAX_ITEM, padding='post', truncating='post')
data_train_val = pad_sequences(sequence_train_val, maxlen=MAX_ITEM, padding='post', truncating='post')
data_test = pad_sequences(sequence_test, maxlen=MAX_ITEM, padding='post', truncating='post')

# extract sequence of items from pandas frame
sequence_train_train = list(map(ast.literal_eval,train_train_baskets["product_id"].tolist()))
sequence_train_val = list(map(ast.literal_eval,train_val_baskets["product_id"].tolist()))

sequence_test = list(map(ast.literal_eval,test_baskets["product_id"].tolist()))

# pad all train and val with max items
data_train_train = pad_sequences(sequence_train_train, maxlen=MAX_ITEM, padding='post', truncating='post')
data_train_val = pad_sequences(sequence_train_val, maxlen=MAX_ITEM, padding='post', truncating='post')

data_test = pad_sequences(sequence_test, maxlen=MAX_ITEM, padding='post', truncating='post')

data_train_train_bs1 = data_train_train[0::3]
data_train_train_bs2 = data_train_train[1::3]
data_train_train_bs3 = data_train_train[2::3]

data_train_val_bs1 = data_train_val[0::3]
data_train_val_bs2 = data_train_val[1::3]
data_train_val_bs3 = data_train_val[2::3]

data_test_bs1 = data_test[0::3]
data_test_bs2 = data_test[1::3]
data_test_bs3 = data_test[2::3]

label_sequence_train_train = list(map(ast.literal_eval,train_train["product_id"].tolist()))
label_sequence_train_val = list(map(ast.literal_eval,train_val["product_id"].tolist()))

data_train_train_one_user = np.concatenate([data_train_train_bs1,data_train_train_bs2,data_train_train_bs3], axis = 1)

mlb = MultiLabelBinarizer(classes=products["product_id"].tolist(), sparse_output=True)

train_train_label = mlb.fit_transform(label_sequence_train_train)
train_val_label = mlb.fit_transform(label_sequence_train_val)

weight_val = np.ones(MAX_ITEM)

## Create a mask for cross entropy

extract all the purchase history of each user

In [ ]:
# create user buying history
prior_sequence = list(map(ast.literal_eval,prior["product_id"].tolist()))

prior_usr = prior["user_id"]

usr_history = {k: [] for k in set(prior_usr)}  

for i in range(len(prior_usr)):
    usr_history[prior_usr[i]].extend(prior_sequence[i])

usr_history_encode = [usr_history[x] for x in range(1, len(usr_history)+1)]

del usr_history

# create user list for train, val and test
mlb = MultiLabelBinarizer(classes=range(1,49689), sparse_output=True)
usr_history = mlb.fit_transform(usr_history_encode)

train_train_usr = train_train_baskets.ix[0::3, "user_id"].values
train_val_usr = train_val_baskets.ix[0::3, "user_id"].values
test_usr = test_baskets.ix[0::3, "user_id"].values

In [ ]:
usr_history.shape

len(train_train_usr)

data_train_train_bs1.shape

train_train_label.shape

Delete some data not used - for the purpose of saving memory

In [ ]:
del prior
del train
del test

# create user_id dict
del train_usr_dict
del test_usr_dict

# split train and test prior data
del prior_train
del prior_test

In [ ]:
data_train_train.shape

## Define the model graph

In [ ]:
EMBEDDING_DIM = 100

num_products = len(products)+1

In [ ]:
# define the model graph
product_embedding_layer = Embedding(input_dim=num_products,
        output_dim=EMBEDDING_DIM,
        embeddings_initializer='normal',
        mask_zero=True,
        input_length=3*MAX_ITEM,
        trainable=True)

# user_mask_layer = Embedding(input_dim=206209,
#         output_dim=49688,
#         weights=[usr_history],
#         mask_zero=True,
#         input_length=1,
#         trainable=False)

lstm_layer = LSTM(100)

bs_1 = Input(shape=(3*MAX_ITEM,), dtype='int32')
# bs_2 = Input(shape=(MAX_ITEM,), dtype='int32')
# bs_3 = Input(shape=(MAX_ITEM,), dtype='int32')
b_hist = Input(shape=(49688,), dtype='float32')


embedded_bs_1 = product_embedding_layer(bs_1)
lstm_out = lstm_layer(embedded_bs_1)
# embedded_bs_2 = product_embedding_layer(bs_2)
# embedded_bs_3 = product_embedding_layer(bs_3)
# embedded_b_hist = user_mask_layer(u_id)

# def embedding_mean(e):
#     return K.mean(e, axis = 1)

# embedded_bs_1_mean = Lambda(embedding_mean)(embedded_bs_1)
# embedded_bs_2_mean = Lambda(embedding_mean)(embedded_bs_2)
# embedded_bs_3_mean = Lambda(embedding_mean)(embedded_bs_3)

# merged = concatenate([embedded_bs_1_mean, embedded_bs_2_mean, embedded_bs_3_mean])
merged = BatchNormalization()(lstm_out)
merged = Dropout(rate_drop_dense)(merged)
merged = Dense(num_dense, kernel_initializer='normal', activation="relu")(merged)

preds = Dense(num_products-1, kernel_initializer='normal', activation='sigmoid')(merged)
mask_preds = merge([preds, b_hist], mode='mul')

In [ ]:
# compile model 
model = Model(inputs=[bs_1, b_hist], outputs=mask_preds)

model.compile(loss='binary_crossentropy', \
        optimizer='adam')
print(STAMP)

In [ ]:
model.summary()

In [ ]:
# set early stopping
early_stopping =EarlyStopping(monitor='val_loss', patience=3)  
bst_model_path = OUTOUT_DIR + STAMP + '.h5' 
model_checkpoint = ModelCheckpoint(bst_model_path, save_best_only=True, save_weights_only=True)

In [ ]:
# train the model
def batch_generator(data_train_train_one_user, user_dict, buy_history, label_mat, batch_size):
    N = np.shape(label_mat)[0]
    number_of_batches = N/batch_size
    counter=0
    shuffle_index = np.arange(N)
    np.random.shuffle(shuffle_index)
    while True:
        index_batch = shuffle_index[batch_size*counter:batch_size*(counter+1)]
        data_input = data_train_train_one_user[index_batch, :]
        u_id = user_dict[index_batch]
        b_hist_input = buy_history[u_id,:].todense()
        label_input = np.asarray(label_mat[index_batch].todense())
        counter += 1
        yield([data_input, b_hist_input],label_input)
        if (counter < number_of_batches):
            np.random.shuffle(shuffle_index)
            counter=0

In [ ]:
hist = model.fit_generator(generator=batch_generator(
                    data_train_train_one_user[0:1000], train_train_usr,\
                    usr_history, train_train_label[0:1000], 20), \
#                     validation_data=([data_train_train_bs1[0:1000], data_train_train_bs2[0:1000], 
#                                       data_train_train_bs3[0:1000]],  train_train_label[0:1000].todense()), \
                    nb_epoch=20, steps_per_epoch=np.shape(train_train_label[0:1000])[0]/20, \
#                     callbacks=[early_stopping, model_checkpoint],
                   )

In [ ]:
# model.load_weights(bst_model_path) # sotre model parameters in .h5 file
bst_val_score = min(hist.history['val_loss'])

usr_history[train_train_usr[0:1000],:].todense()

# make the prediction
print('Start making the submission before fine-tuning')
preds = model.predict([data_train_train_one_user[0:1000], usr_history[train_train_usr[0:1000],:].todense()], batch_size=128, verbose=1)

In [102]:
preds

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]], dtype=float32)

In [103]:
sort_preds = np.argsort(preds)

In [104]:
sort_preds

array([[    0, 33101, 33102, ..., 24851, 47765, 13175],
       [    0, 33112, 33113, ..., 21136, 47765, 21902],
       [    0, 33116, 33117, ...,  8423, 45006, 21902],
       ..., 
       [    0, 33103, 33104, ..., 13175, 21136, 24851],
       [    0, 33113, 33114, ...,   280, 13589, 33197],
       [    0, 33106, 33107, ..., 39274, 33547, 24851]])

In [105]:
trueth = np.array(train_train_label[0:1000].todense())

In [106]:
(trueth * preds).sum(axis = 1)

array([  0.00000000e+00,   0.00000000e+00,   3.05322826e-01,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         3.17387938e-01,   1.12930842e-01,   2.66003668e-01,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   3.93589675e-01,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   6.18960410e-02,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         1.02033801e-01,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         4.89851326e-01,   3.57578173e-02,   0.00000000e+00,
         0.00000000e+00,   9.47058320e-01,   0.00000000e+00,
         1.99050546e-01,   0.00000000e+00,   1.18053926e-01,
         0.00000000e+00,

In [107]:
np.argsort(preds)

array([[    0, 33101, 33102, ..., 24851, 47765, 13175],
       [    0, 33112, 33113, ..., 21136, 47765, 21902],
       [    0, 33116, 33117, ...,  8423, 45006, 21902],
       ..., 
       [    0, 33103, 33104, ..., 13175, 21136, 24851],
       [    0, 33113, 33114, ...,   280, 13589, 33197],
       [    0, 33106, 33107, ..., 39274, 33547, 24851]])

In [108]:
np.max(preds)

0.91256803